In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import importlib
import os
import matplotlib.patches as patches
import dask
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SGECluster
import socket
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, MaxPool2D, Conv2D, Flatten, Dropout, BatchNormalization

In [2]:
hostname = socket.gethostname()
n_workers = 1
n_cores = 4

wks2 = "wn-wks2.fe.hhi.de"
gpu1 = "wn-gpu1.fe.hhi.de"
gpu2 = "wn-gpu-104-01.fe.hhi.de"

if hostname == wks2:
    path = "/data/cluster/projects/infineon-radar/daq_x-har/3_Walking_converted/recording-2020-01-28_11-31-55"
    mem = "20G"      # Allocated memory is critical. For this example it must be at least 16GB 
    q = "wn-37.q"    # Check current queue status on https://hpc-management.fe.hhi.de/wn/phpqstat/
    
    cluster = SGECluster(n_workers=n_workers, cores=n_cores, memory=mem,
                        resource_spec=f"h_vmem={mem}", host=hostname, queue=q,
                         job_extra=["-v MKL_NUM_THREADS=1,NUMEXPR_NUM_THREADS=1,OMP_NUM_THREADS=1"])
elif hostname in (gpu1, gpu2):
    path = os.getcwd() + "/data"
    os.environ["CUDA_VISIBLE_DEVICES"]="3"  # Check current status with nvidia-smi and pick GPU from 0-3
    cluster = LocalCluster(n_workers=n_workers, threads_per_worker=n_cores, host=hostname)
else:
    raise ValueError(f"{hostname} is not a supported host. Please run this example on {wks}, {gpu1} or {gpu2}.")
    

    
client = Client(cluster)
client.wait_for_workers(n_workers=n_workers)
client

Client Scheduler: tcp://127.0.0.1:33537 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 540.71 GB


In [3]:
def split_dataset(dataset: tf.data.Dataset, validation_data_fraction: float):

    validation_data_percent = round(validation_data_fraction * 100)
    if not (0 <= validation_data_percent <= 100):
        raise ValueError("validation data fraction must be ∈ [0,1]")

    dataset = dataset.enumerate()
    train_dataset = dataset.filter(lambda f, data: f % 100 >= validation_data_percent)
    validation_dataset = dataset.filter(lambda f, data: f % 100 < validation_data_percent)

    # remove enumeration
    train_dataset = train_dataset.map(lambda f, data: data)
    validation_dataset = validation_dataset.map(lambda f, data: data)

    return train_dataset, validation_dataset

def data_prep(path):
    dataset_raw = np.load(path, allow_pickle=True)
    arrays = np.array(dataset_raw[()]["data"])
    arrays = arrays - arrays.min()
    arrays = arrays/arrays.max()
    arrays -= arrays.mean()
    arrays = arrays/arrays.std()
    if np.isnan(arrays).any() or np.isinf(arrays).any():
        raise "data have imperfections"
    print(arrays.shape)
    labels = dataset_raw[()]["label"]
    labels = np.array([x-np.array(list(set(labels))).min() for x in labels])
    print(labels.shape)
    return (arrays, labels)

def load_data(path):
    data, label = data_prep(path)
    dataset = tf.data.Dataset.from_tensor_slices((data, label))
    dataset = dataset.shuffle(100000)
    train_dataset, rest = split_dataset(dataset, 0.3)
    test_dataset, valid_dataset = split_dataset(rest, 0.5)
    train_data = train_dataset.shuffle(1000).batch(10)
    valid_data = valid_dataset.batch(10)
    test_data = test_dataset.batch(10)
    return train_data, valid_data, test_data


def make_model():
    model = tf.keras.Sequential([
    Input((1,30,30)),
    Conv2D(filters = 8, kernel_size=(3,3), padding="same", activation="relu", name="c1", data_format="channels_first"),
    Conv2D(filters = 16, kernel_size=(3,3), padding="same", activation="relu", name="c2", data_format="channels_first"),
    MaxPool2D(pool_size=(2,2), strides=(1,1),padding="same", name="m1", data_format="channels_first"),
    
    Conv2D(filters = 16, kernel_size=(3,3), padding="same", activation="relu", name="c3", data_format="channels_first"),
    MaxPool2D(pool_size=(2,2), strides=(1,1),padding="same", name="m2",data_format="channels_first"),
    
    Flatten(),
    Dense(256, activation="relu", use_bias=True),
    Dense(5, activation="softmax", use_bias=True)])
    return model



In [4]:
train_data, valid_data, test_data = load_data("/home/fe/khodabakhshandeh/Projects/radar/radar-ml/Python/data/Config G/box_data.npy")

(59003, 1, 30, 30)
(59003,)


In [119]:
model = make_model()
print(model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.fit(train_data, verbose=1, validation_data=valid_data, epochs=20)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 8, 30, 30)         80        
_________________________________________________________________
c2 (Conv2D)                  (None, 16, 30, 30)        1168      
_________________________________________________________________
m1 (MaxPooling2D)            (None, 16, 30, 30)        0         
_________________________________________________________________
c3 (Conv2D)                  (None, 16, 30, 30)        2320      
_________________________________________________________________
m2 (MaxPooling2D)            (None, 16, 30, 30)        0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 14400)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 256)             

KeyboardInterrupt: 

In [21]:
model = tf.keras.models.load_model("base_model_2")


In [22]:
data, label = data_prep("/home/fe/khodabakhshandeh/Projects/radar/radar-ml/Python/data/Config G/box_data.npy")
model.evaluate(data, label)

(59003, 1, 30, 30)
(59003,)
59003/59003 [==============================] - 2s 39us/sample - loss: 0.0940 - accuracy: 0.9682


[0.09398601971237923, 0.96822196]

In [7]:
def select_random_data(train_data):
    train_data.shuffle(1000)
    _, data = split_dataset(train_data, 0.1)
    return data

def UCB1(mu, n, t):
    P = np.sqrt(2*np.log10(t)/n)
    P = P
    print("p:", P)
    index = np.argmax(np.add(mu, P))
    return index

In [41]:
T = 1024
W = model.layers[-2].get_weights()
n = np.zeros(256)
mu = np.zeros_like(n)
threshold = 0.005
norm_const=0.01
for i in range(1, T):
    #select random train data for comparison
    data = select_random_data(train_data)
    
    #selecting index exploration/exploitation
    if np.where(n==0)[0].size == 0:
        index = UCB1(mu, n, i)
    else:
        index = np.where(n==0)[0][0]
    
    print("index:", i)
    
    #evaluating main model
    loss_base = model.evaluate(data, verbose=0)[0]
    print("/n")

    
    #setting selected node to zero and evaluating again
    W_ = np.copy(W)
    W_[0][:, index] = 0
    model.layers[-2].set_weights(W_)
    loss = model.evaluate(data, verbose=0)[0]
    print("/n")
    
    #calculating delta and reward
    delta = loss_base - loss
    reward = max(0, threshold + delta)/norm_const
    print("reward:", reward)
    
    #updating number of visiting the node and the average reward
    n[index] = n[index]+1
    mu[index] = ((n[index]-1)/n[index])*mu[index] + (1/n[index])*reward
    #initializing the layer to the original trained weights for next round
    model.layers[-2].set_weights(W)
    
    
    

index: 1
/n
/n
reward: 0.0
index: 2
/n
/n
reward: 0.0
index: 3
/n
/n
reward: 1.4826878706751696
index: 4
/n
/n
reward: 1.2053589566198977
index: 5
/n
/n
reward: 1.6420272029265905
index: 6
/n
/n
reward: 2.420802870716301
index: 7
/n
/n
reward: 0.0
index: 8
/n
/n
reward: 0.0
index: 9
/n
/n
reward: 0.08784424497328648
index: 10
/n
/n
reward: 3.323539868060942
index: 11
/n
/n
reward: 1.0660939547829342
index: 12
/n
/n
reward: 0.0
index: 13
/n
/n
reward: 3.1227014902630823
index: 14
/n
/n
reward: 3.206372092140685
index: 15
/n
/n
reward: 0.0
index: 16
/n
/n
reward: 0.0
index: 17
/n
/n
reward: 0.08874890851694418
index: 18
/n
/n
reward: 3.0085838946307364
index: 19
/n
/n
reward: 0.3294646792813489
index: 20
/n
/n
reward: 0.0
index: 21
/n
/n
reward: 0.3508012154815383
index: 22
/n
/n
reward: 2.9824416612578504
index: 23
/n
/n
reward: 1.1203153634712744
index: 24
/n
/n
reward: 0.6663768813471356
index: 25
/n
/n
reward: 6.520653350787514
index: 26
/n
/n
reward: 0.0
index: 27
/n
/n
reward: 2.59

/n
/n
reward: 3.0736104732468013
index: 227
/n
/n
reward: 0.0
index: 228
/n
/n
reward: 6.342963918333943
index: 229
/n
/n
reward: 3.54646722617603
index: 230
/n
/n
reward: 0.9705994682652649
index: 231
/n
/n
reward: 0.0
index: 232
/n
/n
reward: 0.0
index: 233
/n
/n
reward: 2.054647442840418
index: 234
/n
/n
reward: 12.706821001711354
index: 235
/n
/n
reward: 1.9754175288336668
index: 236
/n
/n
reward: 4.240216422648679
index: 237
/n
/n
reward: 0.0
index: 238
/n
/n
reward: 0.0
index: 239
/n
/n
reward: 5.235811154047642
index: 240
/n
/n
reward: 0.0
index: 241
/n
/n
reward: 3.1095835084007035
index: 242
/n
/n
reward: 1.5580563545227184
index: 243
/n
/n
reward: 0.0
index: 244
/n
/n
reward: 2.8154930415607566
index: 245
/n
/n
reward: 0.0
index: 246
/n
/n
reward: 0.0
index: 247
/n
/n
reward: 0.470435173738581
index: 248
/n
/n
reward: 1.0760650691531977
index: 249
/n
/n
reward: 6.220651504539309
index: 250
/n
/n
reward: 0.0
index: 251
/n
/n
reward: 2.951432631129302
index: 252
/n
/n
reward: 1

/n
/n
reward: 0.5607437463033822
p: [2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397 2.19771397
 2.19771397 2.19771397 2.1

/n
/n
reward: 0.0
p: [2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989 2.19997989
 2.19997989 2.19997989 2.19997989 1.55562

/n
/n
reward: 0.0
p: [2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781 2.20221781
 2.20221781 2.20221781 2.20221781 1.55720

...

/n
/n
reward: 0.0
p: [2.4512533  2.4512533  1.0962338  1.41523175 1.41523175 1.41523175
 2.4512533  2.4512533  1.0962338  0.86664892 1.41523175 2.4512533
 0.70761588 1.0962338  2.4512533  2.4512533  1.73329783 1.41523175
 0.92648666 2.4512533  1.73329783 1.00071997 0.86664892 1.41523175
 0.92648666 2.4512533  0.73908068 1.41523175 1.73329783 1.73329783
 2.4512533  0.86664892 1.41523175 0.77515436 1.00071997 1.41523175
 1.73329783 2.4512533  2.4512533  1.41523175 2.4512533  1.73329783
 1.41523175 1.73329783 2.4512533  1.22562665 0.92648666 0.92648666
 2.4512533  2.4512533  2.4512533  1.41523175 1.73329783 1.22562665
 0.92648666 2.4512533  1.73329783 2.4512533  2.4512533  1.41523175
 2.4512533  2.4512533  2.4512533  2.4512533  2.4512533  2.4512533
 2.4512533  1.41523175 0.86664892 0.86664892 0.86664892 2.4512533
 2.4512533  1.22562665 2.4512533  2.4512533  2.4512533  2.4512533
 2.4512533  2.4512533  2.4512533  2.4512533  0.81708443 2.4512533
 1.0962338  0.92648666 1.73329783 0.86664892 0

/n
/n
reward: 1.6983339843295804
p: [2.45177872 2.45177872 1.09646878 1.4155351  1.4155351  1.4155351
 2.45177872 2.45177872 1.09646878 0.86683468 1.4155351  2.45177872
 0.70776755 1.09646878 2.45177872 2.45177872 1.73366936 1.4155351
 0.92668525 2.45177872 1.73366936 1.00093447 0.86683468 1.4155351
 0.92668525 2.45177872 0.65526543 1.4155351  1.73366936 1.73366936
 2.45177872 0.86683468 1.4155351  0.77532051 1.00093447 1.4155351
 1.73366936 2.45177872 2.45177872 1.4155351  2.45177872 1.73366936
 1.4155351  1.73366936 2.45177872 1.22588936 0.92668525 0.92668525
 2.45177872 2.45177872 2.45177872 1.4155351  1.73366936 1.22588936
 0.92668525 2.45177872 1.73366936 2.45177872 2.45177872 1.4155351
 2.45177872 2.45177872 2.45177872 2.45177872 2.45177872 2.45177872
 2.45177872 1.4155351  0.86683468 0.86683468 0.86683468 2.45177872
 2.45177872 1.22588936 2.45177872 2.45177872 2.45177872 2.45177872
 2.45177872 2.45177872 2.45177872 2.45177872 0.81725957 2.45177872
 1.09646878 0.92668525 1.733669

/n
/n
reward: 3.8346565564473347
p: [2.45230247 2.45230247 1.09670301 1.41583749 1.41583749 1.41583749
 2.45230247 2.45230247 1.09670301 0.86701985 1.41583749 2.45230247
 0.70791875 1.09670301 2.45230247 2.45230247 1.73403971 1.41583749
 0.92688321 2.45230247 1.73403971 1.00114829 0.86701985 1.41583749
 0.92688321 2.45230247 0.6554054  1.41583749 1.73403971 1.73403971
 2.45230247 0.86701985 1.41583749 0.77548613 1.00114829 1.41583749
 1.73403971 2.45230247 2.45230247 1.41583749 2.45230247 1.73403971
 1.41583749 1.73403971 2.45230247 1.22615124 0.92688321 0.86701985
 2.45230247 2.45230247 2.45230247 1.41583749 1.73403971 1.22615124
 0.92688321 2.45230247 1.73403971 2.45230247 2.45230247 1.41583749
 2.45230247 2.45230247 2.45230247 2.45230247 2.45230247 2.45230247
 2.45230247 1.41583749 0.86701985 0.86701985 0.86701985 2.45230247
 2.45230247 1.22615124 2.45230247 2.45230247 2.45230247 2.45230247
 2.45230247 2.45230247 2.45230247 2.45230247 0.73939702 2.45230247
 1.09670301 0.92688321 1.7

/n
/n
reward: 0.0
p: [2.45282457 2.45282457 1.09693649 1.41613893 1.41613893 1.41613893
 2.45282457 2.45282457 1.09693649 0.86720444 1.41613893 2.45282457
 0.70806946 1.09693649 2.45282457 2.45282457 1.73440889 1.41613893
 0.92708055 2.45282457 1.73440889 1.00136144 0.86720444 1.41613893
 0.92708055 2.45282457 0.65554494 1.41613893 1.73440889 1.73440889
 2.45282457 0.86720444 1.41613893 0.77565123 1.00136144 1.41613893
 1.73440889 2.45282457 2.45282457 1.41613893 2.45282457 1.73440889
 1.41613893 1.73440889 2.45282457 1.22641228 0.92708055 0.86720444
 2.45282457 2.45282457 2.45282457 1.41613893 1.73440889 1.22641228
 0.92708055 2.45282457 1.73440889 2.45282457 2.45282457 1.41613893
 2.45282457 2.45282457 2.45282457 2.45282457 2.45282457 2.45282457
 2.45282457 1.41613893 0.86720444 0.86720444 0.86720444 2.45282457
 2.45282457 1.22641228 2.45282457 2.45282457 2.45282457 2.45282457
 2.45282457 2.45282457 2.45282457 2.45282457 0.65554494 2.45282457
 1.09693649 0.92708055 1.73440889 0.86720

/n
/n
reward: 0.0
p: [2.45334502 2.45334502 1.09716925 1.41643941 1.41643941 1.41643941
 2.45334502 2.45334502 1.09716925 0.86738845 1.41643941 2.45334502
 0.7082197  1.09716925 2.45334502 2.45334502 1.7347769  1.41643941
 0.92727726 2.45334502 1.7347769  1.00157391 0.86738845 1.41643941
 0.92727726 2.45334502 0.65568404 1.41643941 1.7347769  1.7347769
 2.45334502 0.86738845 1.41643941 0.77581582 1.00157391 1.41643941
 1.7347769  2.45334502 2.45334502 1.41643941 2.45334502 1.7347769
 1.41643941 1.7347769  2.45334502 1.22667251 0.92727726 0.86738845
 2.45334502 2.45334502 2.45334502 1.41643941 1.7347769  1.22667251
 0.92727726 2.45334502 1.7347769  2.45334502 2.45334502 1.41643941
 2.45334502 2.45334502 2.45334502 2.45334502 2.45334502 2.45334502
 2.45334502 1.41643941 0.86738845 0.86738845 0.86738845 2.45334502
 2.45334502 1.22667251 2.45334502 2.45334502 2.45334502 2.45334502
 2.45334502 2.45334502 2.45334502 2.45334502 0.61333626 2.45334502
 1.09716925 0.92727726 1.7347769  0.8673884

In [42]:
n

array([ 1.,  1.,  5.,  3.,  3.,  3.,  1.,  1.,  5.,  8.,  3.,  1., 12.,
        5.,  1.,  1.,  2.,  3.,  7.,  1.,  2.,  6.,  8.,  3.,  7.,  1.,
       14.,  3.,  2.,  2.,  1.,  8.,  3., 10.,  6.,  3.,  2.,  1.,  1.,
        3.,  1.,  2.,  3.,  2.,  1.,  4.,  7.,  8.,  1.,  1.,  1.,  3.,
        2.,  4.,  7.,  1.,  2.,  1.,  1.,  3.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  3.,  8.,  8.,  8.,  1.,  1.,  4.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., 16.,  1.,  5.,  7.,  2.,  8.,  7.,  1., 11.,
        1.,  1.,  3.,  7.,  1., 10.,  1.,  1.,  6.,  1.,  2.,  1.,  1.,
        1.,  2.,  1.,  5.,  7.,  1., 10.,  8., 23.,  5.,  6., 10.,  1.,
       11.,  1.,  4.,  1.,  7.,  1.,  1.,  6., 12.,  1.,  1.,  1.,  7.,
        1.,  1.,  1.,  1.,  3.,  1.,  9.,  1.,  1.,  1.,  1.,  3., 11.,
       11.,  3.,  8.,  5.,  1.,  1.,  5., 14., 10.,  1.,  4.,  1.,  4.,
        7., 28.,  1.,  1.,  1., 12., 13.,  1.,  3., 11.,  1.,  4.,  4.,
        4.,  1.,  7.,  5.,  2.,  1.,  3.,  6.,  2.,  2.,  1.,  1

In [52]:
indexes = np.argsort(mu)[-25:]
indexes

array([184, 238, 203, 233,  96, 225, 235, 165, 224, 228,  12, 115, 151,
       194, 222, 125, 150, 161, 162, 227,  26,  82, 112, 157, 207])

In [44]:
data, label = data_prep("/home/fe/khodabakhshandeh/Projects/radar/radar-ml/Python/data/Config G/box_data.npy")

(59003, 1, 30, 30)
(59003,)


In [53]:
model = tf.keras.models.load_model("base_model_2")
model.evaluate(data, label)
W = model.layers[-2].get_weights()
W_ = np.copy(W)
for i in indexes:
    print(i)
    W_[0][:,i] = 0
model.layers[-2].set_weights(W_)
model.evaluate(data, label)

59003/59003 [==============================] - 2s 39us/sample - loss: 0.0940 - accuracy: 0.9682
184
238
203
233
96
225
235
165
224
228
12
115
151
194
222
125
150
161
162
227
26
82
112
157
207
59003/59003 [==============================] - 2s 38us/sample - loss: 0.1434 - accuracy: 0.9545


[0.14337383103942364, 0.9544938]